%load_ext sql is a Jupyter Notebook magic command that loads in sql extension, which allows you the use of SQL within the Jupyter Notebook environment. 

In [2]:
%load_ext sql

In [33]:
%%sql sqlite:///danny.db
-- connect to the db which is a local file

Done.


[]

In [26]:
%%sql 
-- view and confirm all tables have been loaded into the db
SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///danny.db
Done.


name
sales
menu
members


### Case Study Questions

1. What is the total amount each customer spent at the restaurant?

In [57]:
%%sql
SELECT customer_id,
       sum(price) AS Amount
  FROM menu m
       JOIN
       sales s ON m.product_id = s.product_id
 GROUP BY customer_id;

 * sqlite:///danny.db
Done.


customer_id,Amount
A,76
B,74
C,36


2. How many days has each customer visited the restaurant?

In [58]:
%%sql
SELECT customer_id,
       count(DISTINCT (order_date) ) AS visited
  FROM sales
 GROUP BY customer_id;

 * sqlite:///danny.db
Done.


customer_id,visited
A,4
B,6
C,2


3. What was the first item from the menu purchased by each customer?

In [63]:
%%sql
SELECT customer_id,
       first_value(m.product_name) OVER (PARTITION BY customer_id ORDER BY order_date) AS first_item_purchased
  FROM sales s
       JOIN
       menu m ON s.product_id = m.product_id
 GROUP BY customer_id;


 * sqlite:///danny.db
Done.


customer_id,first_item_purchased
A,sushi
B,curry
C,ramen


4. What is the most purchased item on the menu and how many times was it purchased by all customers?

In [67]:
%%sql
SELECT m.product_name,
       count(*) AS most_purchased_item
  FROM sales s
       JOIN
       menu m ON s.product_id = m.product_id
 GROUP BY m.product_name
 ORDER BY most_purchased_item DESC
 LIMIT 1;


 * sqlite:///danny.db
Done.


product_name,most_purchased_item
ramen,8


5. Which item was the most popular for each customer?

In [44]:
%%sql
WITH cte AS (
    SELECT customer_id,
           product_name,
           COUNT( * ) AS purchase_count,
           row_number() OVER (PARTITION BY customer_id ORDER BY COUNT( * ) DESC) AS rank
      FROM sales
           JOIN
           menu USING (
               product_id
           )
     GROUP BY customer_id,
              product_name
)
SELECT customer_id,
       product_name,
       purchase_count
  FROM cte
 WHERE rank = 1;

 * sqlite:///danny.db
Done.


customer_id,product_name,purchase_count
A,ramen,3
B,sushi,2
C,ramen,3


6. Which item was purchased first by the customer after they became a member?

In [9]:
%%sql
WITH cte AS (
    SELECT customer_id,
           product_name,
           order_date,
           join_date,
           row_number() OVER (PARTITION BY customer_id ORDER BY order_date) AS rank
      FROM sales
           JOIN
           menu USING (
               product_id
           )
           JOIN
           members USING (
               customer_id
           )
     WHERE order_date >= join_date
)
SELECT customer_id,
       join_date,
       order_date,
       product_name
  FROM cte
 WHERE rank = 1;


 * sqlite:///danny.db
Done.


customer_id,join_date,order_date,product_name
A,2021-01-07,2021-01-07,curry
B,2021-01-09,2021-01-11,sushi


7. Which item was purchased just before the customer became a member?

In [8]:
%%sql
WITH cte AS (
    SELECT customer_id,
           product_name,
           join_date,
           order_date,
           row_number() OVER (PARTITION BY customer_id ORDER BY order_date DESC) AS rank
      FROM sales
           JOIN
           menu USING (
               product_id
           )
           JOIN
           members USING (
               customer_id
           )
     WHERE order_date < join_date
)
SELECT customer_id,
       product_name,
       join_date,
       order_date
  FROM cte
 WHERE rank = 1

 * sqlite:///danny.db
Done.


customer_id,product_name,join_date,order_date
A,sushi,2021-01-07,2021-01-01
B,sushi,2021-01-09,2021-01-04


8. What is the total items and amount spent for each member before they became a member?

In [7]:
%%sql
    SELECT customer_id,
       count( * ) total_items,
       sum(price) amount_spent
  FROM sales
       JOIN
       menu USING (
           product_id
       )
       JOIN
       members USING (
           customer_id
       )
 WHERE order_date < join_date
 GROUP BY customer_id
 ORDER BY amount_spent DESC;


 * sqlite:///danny.db
Done.


customer_id,total_items,amount_spent
B,3,40
A,2,25


9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?

In [12]:
%%sql
WITH cte AS (
    SELECT customer_id,
           product_name,
           CASE WHEN product_name = 'sushi' THEN price * 20 
           ELSE price * 10 END AS points
      FROM sales
           JOIN
           menu USING (
               product_id
           )
)
SELECT customer_id,
       sum(points) AS points
  FROM cte
 GROUP BY customer_id;


 * sqlite:///danny.db
Done.


customer_id,points
A,860
B,940
C,360


10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

In [46]:
%%sql
WITH cte AS (
    SELECT
        customer_id,
        order_date,
        join_date,
        product_name,
        price,
        CASE
            WHEN order_date BETWEEN join_date and (select date(join_date,'+6 days')) THEN price * 20
            --mysql altervative uses DATEADD instead of select date
            When product_name = 'suchi' then price * 20
            ELSE price * 10
        END AS test
    FROM
        sales
    JOIN
        menu USING (product_id)
    JOIN
        members USING (customer_id)
)

SELECT
    customer_id,
    SUM(test) AS total_points
FROM
    cte
WHERE
    strftime('%m', order_date) = '01'
    --alternate DATE_TRUNC('month', order_date) = '2021-01-01'
GROUP BY
    customer_id;


 * sqlite:///danny.db
Done.


customer_id,total_points
A,1270
B,720


Bonus Question1 - Joinig all the things

In [20]:
%%sql

SELECT customer_id,
       order_date,
       product_name,
       price,
       CASE WHEN order_date >= join_date THEN 'Y' ELSE 'N' END AS member
  FROM sales
       LEFT JOIN
       members USING (
           customer_id
       )
       JOIN
       menu USING (
           product_id
       );


 * sqlite:///danny.db
Done.


customer_id,order_date,product_name,price,member
A,2021-01-01,sushi,10,N
A,2021-01-01,curry,15,N
A,2021-01-07,curry,15,Y
A,2021-01-10,ramen,12,Y
A,2021-01-11,ramen,12,Y
A,2021-01-11,ramen,12,Y
B,2021-01-01,curry,15,N
B,2021-01-02,curry,15,N
B,2021-01-04,sushi,10,N
B,2021-01-11,sushi,10,Y


Bonus Question2 - Ranking all things

In [36]:
%%sql
WITH cte AS (
    SELECT customer_id,
           order_date,
           product_name,
           price,
           CASE WHEN order_date >= join_date THEN 'Y' ELSE 'N' END AS member
      FROM sales
           LEFT JOIN
           members USING (
               customer_id
           )
           JOIN
           menu USING (
               product_id
           )
)
SELECT *,
       CASE WHEN member = 'Y' THEN dense_rank() OVER (PARTITION BY customer_id,
           member ORDER BY order_date) ELSE 'NULL' END AS rank 
           
           -- Did you know you can partition on two columns?
  FROM cte;


 * sqlite:///danny.db
Done.


customer_id,order_date,product_name,price,member,rank
A,2021-01-01,sushi,10,N,NULL
A,2021-01-01,curry,15,N,NULL
A,2021-01-07,curry,15,Y,1
A,2021-01-10,ramen,12,Y,2
A,2021-01-11,ramen,12,Y,3
A,2021-01-11,ramen,12,Y,3
B,2021-01-01,curry,15,N,NULL
B,2021-01-02,curry,15,N,NULL
B,2021-01-04,sushi,10,N,NULL
B,2021-01-11,sushi,10,Y,1


This brings us to the end of CASE STUDY 1. See you in the next.